In [1]:
from utils import *
import pymongo

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["db"]
birds = mydb["electric_scooters"]

In [3]:
def get_birds_ids(birds):
    return list(birds.distinct("id"))

In [4]:
birds_ids = get_birds_ids(birds)
print(len(birds_ids))

7642


In [94]:
test_id = birds_ids[300]

def get_total_distance(bird_id, birds, time):
    if time == "any":
        bird_by_id = birds.find({"id": bird_id})
    else:
        #bird_by_id = birds.find({"id": bird_id},{"time": "/2019/"})
        bird_by_id = birds.find({"$and": [{"id": bird_id}, {"time":{"$regex": u"2019"}}]})
    bird_by_id = list(bird_by_id)
    print(len(bird_by_id))
    print(bird_by_id[0])
    total_distance = 0

    for i in range(len(bird_by_id) - 1):
        previous = bird_by_id[i]
        actual = bird_by_id[i + 1]
        #print(calculate_distance(previous.get("location"), actual.get("location")))
        total_distance += calculate_distance(previous.get("location"), actual.get("location"))

    return total_distance

In [95]:
get_total_distance(test_id, birds, "2")

172
{'_id': '5db779c74e423f8ba38c3a33', 'area_key': 'F461754DC547D80D5B8E77A5018FC6E8', 'battery_level': '26', 'captive': 'false', 'code': None, 'estimated_range': '6003', 'id': 'bec01b21-283b-4313-909d-ac46b0e59dc3', 'location': {'latitude': '50.06211833333334', 'longitude': '19.96195'}, 'model': 'rf', 'time': '2019-10-28 23:28:57', 'total_distance': '0', 'vehicle_class': 'scooter'}


1.4932515228610381